# Caderno 12 - Compila todas as métricas em 6 tabelas.

As tabelas são as combinações dos conjuntos de queries (3 conjuntos) e k = [5, 10, 20].

## 1. Pastas com os resultados, nomes dos arquivos de índice. docs/queries/qrels

In [52]:
import pandas as pd

PASTA_DADOS = './dados/'
# A pasta dos JURIS aqui não é a pasta original, e sim o resultado do caderno 1 (os documentos já estão filtrados)
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Cenário atual
PASTA_RESULTADO_PESQUISA_SOLR = f'{PASTA_DADOS}outputs/2_pesquisa_queries_na_base_atual/'

# Cenário BM25 padrão, com enunciado único e com enunciado dobrado
INDICE_BM25_PADRAO = f'{PASTA_DADOS}/outputs/4_metricas_bm25_padrao/indice_js_enunciado_e_excerto.pickle'
INDICE_BM25_PADRAO_ENUNCIADO_DOBRADO = f'{PASTA_DADOS}/outputs/4_metricas_bm25_padrao/indice_js_enunciado_2x_e_excerto.pickle'
# Cenário BM25 padrão com DOC2QUERY
INDICE_BM25_COM_DOC2QUERY_1QUERY = f'{PASTA_DADOS}/outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query_apenas_uma_query.pickle'
INDICE_BM25_COM_DOC2QUERY_5QUERIES = f'{PASTA_DADOS}/outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query.pickle'
# Cenário BM25 padrão com reescrita de enunciado
INDICE_BM25_COM_REESCRITA_ENUNCIADO_GPT = f'{PASTA_DADOS}/outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_gpt.pickle'
INDICE_BM25_COM_REESCRITA_ENUNCIADO_LLAMA = f'{PASTA_DADOS}/outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_llama.pickle'
# Cenário BM25 padrão com sinônimos do enunciado
INDICE_BM25_COM_SINONIMOS_ENUNCIADO_GPT = f'{PASTA_DADOS}/outputs/10_metricas_bm25_com_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_sinonimos_enunciado_gpt.pickle'
INDICE_BM25_COM_SINONIMOS_ENUNCIADO_GPT_4o = f'{PASTA_DADOS}/outputs/10_metricas_bm25_com_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_sinonimos_enunciado_gpt4o.pickle'
INDICE_BM25_COM_SINONIMOS_ENUNCIADO_LLAMA = f'{PASTA_DADOS}/outputs/10_metricas_bm25_com_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_sinonimos_enunciado_llama.pickle'
# Cenário BM25 padrão com DOC2QUERY (5 queries) e sinônimos do enunciado
INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_GPT = f'{PASTA_DADOS}/outputs/11_metricas_bm25_com_doc2query_e_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_doc2query_5q_e_gpt.pickle'
INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_GPT_4o = f'{PASTA_DADOS}/outputs/11_metricas_bm25_com_doc2query_e_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_doc2query_5q_e_gpt4o.pickle'
INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_LLAMA = f'{PASTA_DADOS}/outputs/11_metricas_bm25_com_doc2query_e_sinonimos_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_doc2query_5q_e_llama.pickle'


# Carrega o resultado da pesquisa do Solr
resultado_pesq_solr = pd.read_csv(f'{PASTA_RESULTADO_PESQUISA_SOLR}resultado_solr_pesquisa_original.csv', sep='|')
resultado_pesq_solr = resultado_pesq_solr[resultado_pesq_solr.ENGINE == 'SOLR_PESQUISA_ORIGINAL_selectSwanSynonym']

In [53]:
# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

docs, queries, qrels = carrega_juris_tcu()

## 2. Cria índices e buscadores

In [62]:
from bm25 import IndiceInvertido, BM25, tokenizador_pt_remove_html

# Lista ordenada para exibição
# Apenas inverte a visualização da reescrita com os sinônimos
lista_nome_buscadores = ["atual",
                       "bm25_padrao",
                       "bm25_padrao_enunciado_dobrado",

                       "doc2query_1query",
                       "doc2query_5queries",

                       "sinonimos_enunciado_gpt",
                       "sinonimos_enunciado_gpt_4o",
                       "sinonimos_enunciado_llama",

                       "reescrita_enunciado_gpt",
                       "reescrita_enunciado_llama",

                       "doc2query_5queries_sinonimos_gpt",
                       "doc2query_5queries_sinonimos_gpt_4o",
                       "doc2query_5queries_sinonimos_llama"]

mapa_nome_arquivos_indices = {
    "bm25_padrao": INDICE_BM25_PADRAO,
    "bm25_padrao_enunciado_dobrado": INDICE_BM25_PADRAO_ENUNCIADO_DOBRADO,
    "doc2query_1query": INDICE_BM25_COM_DOC2QUERY_1QUERY,
    "doc2query_5queries": INDICE_BM25_COM_DOC2QUERY_5QUERIES,
    "reescrita_enunciado_gpt": INDICE_BM25_COM_REESCRITA_ENUNCIADO_GPT,
    "reescrita_enunciado_llama": INDICE_BM25_COM_REESCRITA_ENUNCIADO_LLAMA,
    "sinonimos_enunciado_gpt": INDICE_BM25_COM_SINONIMOS_ENUNCIADO_GPT,
    "sinonimos_enunciado_gpt_4o": INDICE_BM25_COM_SINONIMOS_ENUNCIADO_GPT_4o,
    "sinonimos_enunciado_llama": INDICE_BM25_COM_SINONIMOS_ENUNCIADO_LLAMA,
    "doc2query_5queries_sinonimos_gpt": INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_GPT,
    "doc2query_5queries_sinonimos_gpt_4o": INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_GPT_4o,
    "doc2query_5queries_sinonimos_llama": INDICE_BM25_COM_DOC2QUERY_5QUERIES_E_SINONIMOS_ENUNCIADO_LLAMA
}

# Cria os IndiceInvertidos
mapa_indices = {nome: IndiceInvertido(tokenizador_pt_remove_html) for nome, _ in mapa_nome_arquivos_indices.items()}

# Carrega dos pickles
for nome, indice in mapa_indices.items():
    print(f'Carregando indice {nome} com o arquivo {mapa_nome_arquivos_indices[nome]}')
    indice.from_pickle(mapa_nome_arquivos_indices[nome])

# Cria os buscadores
# Sobre a escolha de k1 e b:
# https://www.elastic.co/pt/blog/practical-bm25-part-3-considerations-for-picking-b-and-k1-in-elasticsearch
mapa_buscadores = {nome: BM25(indice, k1=1.2, b=0.75, bias_idf=1) for nome, indice in mapa_indices.items()}

Carregando indice bm25_padrao com o arquivo ./dados//outputs/4_metricas_bm25_padrao/indice_js_enunciado_e_excerto.pickle
Carregando indice bm25_padrao_enunciado_dobrado com o arquivo ./dados//outputs/4_metricas_bm25_padrao/indice_js_enunciado_2x_e_excerto.pickle
Carregando indice doc2query_1query com o arquivo ./dados//outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query_apenas_uma_query.pickle
Carregando indice doc2query_5queries com o arquivo ./dados//outputs/6_metricas_bm25_com_doc2query/indice_js_enunciado_e_excerto_e_doc2query.pickle
Carregando indice reescrita_enunciado_gpt com o arquivo ./dados//outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_gpt.pickle
Carregando indice reescrita_enunciado_llama com o arquivo ./dados//outputs/8_metricas_bm25_com_reescrita_enunciado_gpt_llama/indice_js_enunciado_e_excerto_e_enunciado_reescrito_llama.pickle
Carregando indice sinonimos_enunciado_gpt com o arqu

## 3. Extrai as métricas

In [55]:
from tqdm import tqdm

def pesquisar_usando_buscador(nome, buscador):
    col_resultado_query_key=[]
    col_resultado_doc_key=[]
    col_resultado_rank=[]
    
    for i, row in tqdm(queries.iterrows(), total=len(queries), desc=f'Pesquisando usando {nome}'):
        query_key = row.KEY
        query_text = row.TEXT
        resultados = buscador.pesquisar(query_text)
    
        primeiros_50_docs = [tupla_key_score[0] for tupla_key_score in resultados[:50]]
        queries_keys = [query_key] * len(primeiros_50_docs)
        ranking = list(range(1, len(primeiros_50_docs)+1))
    
        col_resultado_query_key.extend(queries_keys)
        col_resultado_doc_key.extend(primeiros_50_docs)
        col_resultado_rank.extend(ranking)
    
    df_resultados = pd.DataFrame({
        "QUERY_KEY": col_resultado_query_key,
        "DOC_KEY": col_resultado_doc_key,
        "RANK": col_resultado_rank,
    })
    return df_resultados

In [56]:
from metricas import metricas

# Pesquisa em todos os buscadores
mapa_resultados = {nome: pesquisar_usando_buscador(nome, buscador) for nome, buscador in mapa_buscadores.items()}

# Extrai as métricas
mapa_metricas = {nome: metricas(resultados, qrels, aproximacao_trec_eval=True) for nome, resultados in mapa_resultados.items()}
# Insere no mapa de métricas as métricas atuais
mapa_metricas['atual'] = metricas(resultado_pesq_solr, qrels, aproximacao_trec_eval=True)

Pesquisando usando sinonimos_enunciado_llama: 100%|██████████████████████████████████| 150/150 [00:01<00:00, 98.26it/s]
Pesquisando usando doc2query_5queries_sinonimos_gpt: 100%|███████████████████████████| 150/150 [00:01<00:00, 89.56it/s]
Pesquisando usando doc2query_5queries_sinonimos_gpt_4o: 100%|████████████████████████| 150/150 [00:01<00:00, 86.32it/s]
Pesquisando usando doc2query_5queries_sinonimos_llama: 100%|█████████████████████████| 150/150 [00:01<00:00, 85.98it/s]


## 4. Exibe as métricas

In [57]:
# Imprime as métricas para o conjunto de queries 1 (0:50), 2 (100:150), ou 3 (100:150) 
# e para um determinado k (foi gerado para k = 5, 10, 20 e 50.

def compara_metricas(con_query, k):
    # Acumula as métricas
    precisao = []
    recall = []
    mrr = []
    ndcg = []

    for nome in lista_nome_buscadores:
        estatisticas = mapa_metricas[nome][50*(con_query-1):50*(con_query)].describe()
        precisao.append(estatisticas.loc['mean', f'P@{k}'])
        recall.append(estatisticas.loc['mean', f'R@{k}'])
        mrr.append(estatisticas.loc['mean', f'MRR@{k}'])
        ndcg.append(estatisticas.loc['mean', f'nDCG@{k}'])

    df = pd.DataFrame({
        "Modelo": lista_nome_buscadores,
        f"P@{k}": precisao,
        f"R@{k}": recall,
        f"MRR@{k}": mrr,
        f"nDCG@{k}": ndcg
    })
    return df

def compara_metricas_todas_queries(k):
    # Acumula as métricas
    precisao = []
    recall = []
    mrr = []
    ndcg = []

    for nome in lista_nome_buscadores:
        estatisticas = mapa_metricas[nome].describe()
        precisao.append(estatisticas.loc['mean', f'P@{k}'])
        recall.append(estatisticas.loc['mean', f'R@{k}'])
        mrr.append(estatisticas.loc['mean', f'MRR@{k}'])
        ndcg.append(estatisticas.loc['mean', f'nDCG@{k}'])

    df = pd.DataFrame({
        "Modelo": lista_nome_buscadores,
        f"P@{k}": precisao,
        f"R@{k}": recall,
        f"MRR@{k}": mrr,
        f"nDCG@{k}": ndcg
    })
    return df

pd.set_option('display.precision', 4)

In [58]:
for con_query in [1, 2, 3]:
#    for k in [5, 10, 20]:
    for k in [5]:
        print(f'Resultados para conjunto de query {con_query} e k={k}')
        display(compara_metricas(con_query, k))

Resultados para conjunto de query 1 e k=5


,Modelo,P@5,R@5,MRR@5,nDCG@5
0,atual,0.288,0.1155,0.3720,0.2653
1,bm25_padrao,0.272,0.1106,0.5253,0.2824
2,bm25_padrao_enunciado_dobrado,0.300,0.1225,0.5107,0.3061
3,doc2query_1query,0.292,0.1180,0.5187,0.3023
4,doc2query_5queries,0.388,0.1568,0.6417,0.4044
5,sinonimos_enunciado_gpt,0.316,0.1276,0.5387,0.3224
6,sinonimos_enunciado_gpt_4o,0.292,0.1177,0.5357,0.3056
7,sinonimos_enunciado_llama,0.312,0.1255,0.5380,0.3204
8,reescrita_enunciado_gpt,0.280,0.1138,0.5040,0.2837
9,reescrita_enunciado_llama,0.268,0.1090,0.4377,0.2643


Resultados para conjunto de query 2 e k=5


,Modelo,P@5,R@5,MRR@5,nDCG@5
0,atual,0.456,0.1892,0.8667,0.5639
1,bm25_padrao,0.500,0.2077,0.8620,0.5713
2,bm25_padrao_enunciado_dobrado,0.544,0.2273,0.9340,0.6327
3,doc2query_1query,0.496,0.2062,0.8920,0.5807
4,doc2query_5queries,0.556,0.2315,0.8773,0.6235
5,sinonimos_enunciado_gpt,0.528,0.2196,0.8797,0.6015
6,sinonimos_enunciado_gpt_4o,0.528,0.2195,0.9020,0.6025
7,sinonimos_enunciado_llama,0.528,0.2197,0.8690,0.5956
8,reescrita_enunciado_gpt,0.512,0.2135,0.8867,0.5912
9,reescrita_enunciado_llama,0.528,0.2193,0.8940,0.5992


Resultados para conjunto de query 3 e k=5


,Modelo,P@5,R@5,MRR@5,nDCG@5
0,atual,0.036,0.0164,0.1100,0.0510
1,bm25_padrao,0.520,0.2340,0.9150,0.6030
2,bm25_padrao_enunciado_dobrado,0.552,0.2484,0.9400,0.6398
3,doc2query_1query,0.520,0.2338,0.9500,0.6129
4,doc2query_5queries,0.528,0.2374,0.9367,0.6202
5,sinonimos_enunciado_gpt,0.516,0.2317,0.9133,0.6035
6,sinonimos_enunciado_gpt_4o,0.528,0.2386,0.9340,0.6170
7,sinonimos_enunciado_llama,0.512,0.2307,0.9217,0.6044
8,reescrita_enunciado_gpt,0.544,0.2446,0.9400,0.6296
9,reescrita_enunciado_llama,0.556,0.2496,0.9550,0.6398


In [59]:
for con_query in [1, 2, 3]:
#    for k in [5, 10, 20]:
    for k in [10]:
        print(f'Resultados para conjunto de query {con_query} e k={k}')
        display(compara_metricas(con_query, k))

Resultados para conjunto de query 1 e k=10


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.260,0.2120,0.3963,0.2656
1,bm25_padrao,0.238,0.1966,0.5386,0.2753
2,bm25_padrao_enunciado_dobrado,0.286,0.2367,0.5352,0.3147
3,doc2query_1query,0.270,0.2230,0.5406,0.3042
4,doc2query_5queries,0.336,0.2739,0.6539,0.3851
5,sinonimos_enunciado_gpt,0.268,0.2214,0.5507,0.3073
6,sinonimos_enunciado_gpt_4o,0.258,0.2136,0.5557,0.2972
7,sinonimos_enunciado_llama,0.274,0.2266,0.5476,0.3084
8,reescrita_enunciado_gpt,0.262,0.2143,0.5196,0.2881
9,reescrita_enunciado_llama,0.262,0.2146,0.4579,0.2792


Resultados para conjunto de query 2 e k=10


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.288,0.2396,0.8667,0.4579
1,bm25_padrao,0.378,0.3176,0.8665,0.5106
2,bm25_padrao_enunciado_dobrado,0.416,0.3488,0.9394,0.5625
3,doc2query_1query,0.380,0.3190,0.8965,0.5198
4,doc2query_5queries,0.402,0.3380,0.8816,0.5458
5,sinonimos_enunciado_gpt,0.408,0.3428,0.8845,0.5435
6,sinonimos_enunciado_gpt_4o,0.394,0.3297,0.9049,0.5362
7,sinonimos_enunciado_llama,0.394,0.3307,0.8735,0.5297
8,reescrita_enunciado_gpt,0.388,0.3258,0.8958,0.5291
9,reescrita_enunciado_llama,0.396,0.3325,0.8940,0.5323


Resultados para conjunto de query 3 e k=10


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.018,0.0164,0.1100,0.0372
1,bm25_padrao,0.388,0.3451,0.9175,0.5328
2,bm25_padrao_enunciado_dobrado,0.418,0.3726,0.9425,0.5673
3,doc2query_1query,0.392,0.3485,0.9525,0.5435
4,doc2query_5queries,0.408,0.3616,0.9389,0.5561
5,sinonimos_enunciado_gpt,0.406,0.3605,0.9153,0.5461
6,sinonimos_enunciado_gpt_4o,0.408,0.3631,0.9340,0.5523
7,sinonimos_enunciado_llama,0.396,0.3524,0.9237,0.5409
8,reescrita_enunciado_gpt,0.406,0.3601,0.9425,0.5554
9,reescrita_enunciado_llama,0.412,0.3655,0.9550,0.5622


In [60]:
for con_query in [1, 2, 3]:
#    for k in [5, 10, 20]:
    for k in [20]:
        print(f'Resultados para conjunto de query {con_query} e k={k}')
        display(compara_metricas(con_query, k))

Resultados para conjunto de query 1 e k=20


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,atual,0.218,0.3616,0.4043,0.3415
1,bm25_padrao,0.209,0.3473,0.5446,0.3525
2,bm25_padrao_enunciado_dobrado,0.224,0.3704,0.5416,0.3788
3,doc2query_1query,0.220,0.3635,0.5464,0.3731
4,doc2query_5queries,0.253,0.4142,0.6586,0.4466
5,sinonimos_enunciado_gpt,0.220,0.3648,0.5563,0.3786
6,sinonimos_enunciado_gpt_4o,0.215,0.3563,0.5586,0.3702
7,sinonimos_enunciado_llama,0.221,0.3663,0.5508,0.3781
8,reescrita_enunciado_gpt,0.213,0.3524,0.5274,0.3574
9,reescrita_enunciado_llama,0.215,0.3568,0.4652,0.3494


Resultados para conjunto de query 2 e k=20


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,atual,0.157,0.2611,0.8667,0.4457
1,bm25_padrao,0.271,0.4538,0.8665,0.5618
2,bm25_padrao_enunciado_dobrado,0.282,0.4716,0.9394,0.6017
3,doc2query_1query,0.277,0.4635,0.8965,0.5768
4,doc2query_5queries,0.286,0.4783,0.8816,0.5951
5,sinonimos_enunciado_gpt,0.283,0.4723,0.8845,0.5884
6,sinonimos_enunciado_gpt_4o,0.270,0.4501,0.9049,0.5772
7,sinonimos_enunciado_llama,0.275,0.4594,0.8735,0.5773
8,reescrita_enunciado_gpt,0.276,0.4623,0.8958,0.5791
9,reescrita_enunciado_llama,0.282,0.4717,0.8958,0.5829


Resultados para conjunto de query 3 e k=20


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,atual,0.009,0.0164,0.1100,0.0364
1,bm25_padrao,0.269,0.4762,0.9175,0.5867
2,bm25_padrao_enunciado_dobrado,0.277,0.4891,0.9425,0.6124
3,doc2query_1query,0.273,0.4824,0.9525,0.5995
4,doc2query_5queries,0.278,0.4909,0.9389,0.6097
5,sinonimos_enunciado_gpt,0.273,0.4840,0.9153,0.5945
6,sinonimos_enunciado_gpt_4o,0.273,0.4840,0.9340,0.6004
7,sinonimos_enunciado_llama,0.274,0.4851,0.9237,0.5988
8,reescrita_enunciado_gpt,0.275,0.4862,0.9425,0.6064
9,reescrita_enunciado_llama,0.285,0.5047,0.9550,0.6194


In [61]:
for k in [5, 10, 20]:
    display(compara_metricas_todas_queries(k))

,Modelo,P@5,R@5,MRR@5,nDCG@5
0,atual,0.2600,0.1070,0.4496,0.2934
1,bm25_padrao,0.4307,0.1841,0.7674,0.4856
2,bm25_padrao_enunciado_dobrado,0.4653,0.1994,0.7949,0.5262
3,doc2query_1query,0.4360,0.1860,0.7869,0.4986
4,doc2query_5queries,0.4907,0.2086,0.8186,0.5494
5,sinonimos_enunciado_gpt,0.4533,0.1930,0.7772,0.5091
6,sinonimos_enunciado_gpt_4o,0.4493,0.1919,0.7906,0.5083
7,sinonimos_enunciado_llama,0.4507,0.1919,0.7762,0.5068
8,reescrita_enunciado_gpt,0.4453,0.1906,0.7769,0.5015
9,reescrita_enunciado_llama,0.4507,0.1926,0.7622,0.5011


,Modelo,P@10,R@10,MRR@10,nDCG@10
0,atual,0.1887,0.1560,0.4576,0.2536
1,bm25_padrao,0.3347,0.2864,0.7742,0.4396
2,bm25_padrao_enunciado_dobrado,0.3733,0.3194,0.8057,0.4815
3,doc2query_1query,0.3473,0.2968,0.7965,0.4558
4,doc2query_5queries,0.3820,0.3245,0.8248,0.4957
5,sinonimos_enunciado_gpt,0.3607,0.3082,0.7835,0.4656
6,sinonimos_enunciado_gpt_4o,0.3533,0.3021,0.7982,0.4619
7,sinonimos_enunciado_llama,0.3547,0.3033,0.7816,0.4597
8,reescrita_enunciado_gpt,0.3520,0.3001,0.7860,0.4575
9,reescrita_enunciado_llama,0.3567,0.3042,0.7690,0.4579


,Modelo,P@20,R@20,MRR@20,nDCG@20
0,atual,0.1280,0.2130,0.4603,0.2746
1,bm25_padrao,0.2497,0.4258,0.7762,0.5004
2,bm25_padrao_enunciado_dobrado,0.2610,0.4437,0.8078,0.5310
3,doc2query_1query,0.2567,0.4365,0.7985,0.5165
4,doc2query_5queries,0.2723,0.4611,0.8263,0.5505
5,sinonimos_enunciado_gpt,0.2587,0.4404,0.7854,0.5205
6,sinonimos_enunciado_gpt_4o,0.2527,0.4301,0.7991,0.5159
7,sinonimos_enunciado_llama,0.2567,0.4369,0.7827,0.5181
8,reescrita_enunciado_gpt,0.2547,0.4336,0.7886,0.5143
9,reescrita_enunciado_llama,0.2607,0.4444,0.7720,0.5172
